In [ ]:
# %pip install -U ipywidgets widgetsnbextension huggingface_hub tqdm


In [ ]:
import os

# Tắt thanh tiến trình của huggingface_hub để tránh lỗi ipywidgets
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"


In [3]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from underthesea import sent_tokenize
from transformers import AutoTokenizer
from tqdm.notebook import tqdm

tqdm.pandas()


In [4]:
# Thêm progress_apply vào pandas để có thanh tiến trình
tqdm.pandas()


# %pip install underthesea


# --- 1. Tải model retriever ---

In [ ]:
# --- 1. Tải các model cần thiết ---
# Model để chấm điểm câu (giống model cũ của bạn)
scorer_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

# Tokenizer của model chính bạn sẽ huấn luyện (phải cùng loại)
# Ví dụ: dùng tokenizer của model trong file config của bạn
main_tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")



In [ ]:
MAX_TOKENS = 200  # Giới hạn của model chính


In [ ]:
def summarize_and_truncate(context: str, scorer, tokenizer, max_tokens: int) -> str:
    """
    Tóm tắt trích rút context để vừa với giới hạn token mà vẫn giữ thứ tự.
    """
    # Bước 1: Tách câu
    sentences = sent_tokenize(context)

    # Nếu context đã đủ ngắn thì trả về luôn
    if len(tokenizer.encode(context)) <= max_tokens:
        return context

    # Bước 2: Vector hóa
    # Vector trung tâm đại diện cho toàn bộ văn bản
    context_embedding = scorer.encode(context, convert_to_tensor=True)
    # Vector cho từng câu
    sentence_embeddings = scorer.encode(sentences, convert_to_tensor=True)

    # Bước 3: Chấm điểm
    cosine_scores = util.cos_sim(context_embedding, sentence_embeddings)[0]

    # Bước 4: Tạo danh sách (điểm số, chỉ số vị trí, câu)
    scored_sentences = []
    for i, score in enumerate(cosine_scores):
        scored_sentences.append((score.item(), i, sentences[i]))

    # Sắp xếp theo điểm số từ cao xuống thấp
    scored_sentences.sort(key=lambda x: x[0], reverse=True)

    # Bước 5: Xây dựng lại context
    final_sentences = []
    current_tokens = 0

    # Lấy các câu quan trọng nhất theo điểm số
    for score, index, sentence in scored_sentences:
        final_sentences.append((index, sentence))

    # Sắp xếp lại các câu đã chọn theo thứ tự gốc
    final_sentences.sort(key=lambda x: x[0])

    # Ghép câu lại cho đến khi đạt giới hạn token
    summarized_context = ""
    for index, sentence in final_sentences:
        temp_context = (
            summarized_context + " " + sentence if summarized_context else sentence
        )
        if len(tokenizer.encode(temp_context)) > max_tokens:
            break
        summarized_context = temp_context

    return summarized_context


# --- 2. Đọc dữ liệu gốc ---

In [ ]:
# --- Áp dụng vào DataFrame ---
df = pd.read_csv("data/vihallu-train.csv")
df["context"] = df["context"].astype(str)


In [17]:
print("Bắt đầu tóm tắt và rút gọn context...")
# Áp dụng hàm mới
df["summarized_context"] = df["context"].progress_apply(
    lambda x: summarize_and_truncate(x, scorer_model, main_tokenizer, MAX_TOKENS)
)
print("✅ Xử lý thành công.")

# Lưu file mới
output_path = "data/vihallu-train-summarized.csv"
# Bạn có thể giữ lại cột context gốc và cột đã xử lý để so sánh
df[["prompt", "response", "label", "context", "summarized_context"]].to_csv(
    output_path, index=False, encoding="utf-8-sig"
)
print(f"✅ Đã lưu dữ liệu đã xử lý vào: {output_path}")


Bắt đầu tóm tắt và rút gọn context...


  0%|          | 0/7000 [00:00<?, ?it/s]

✅ Xử lý thành công.
✅ Đã lưu dữ liệu đã xử lý vào: data/vihallu-train-summarized.csv


In [18]:
context = df["context"].iloc[0]
summarized_context = df["summarized_context"].iloc[0]

print("Context gốc:")
print(context)
print("\nContext đã tóm tắt và rút gọn:")
print(summarized_context)

cosine_score = util.cos_sim(
    scorer_model.encode(context, convert_to_tensor=True),
    scorer_model.encode(summarized_context, convert_to_tensor=True),
)[0][0]
print(f"\nĐiểm cosine similarity giữa context gốc và đã tóm tắt: {cosine_score:.4f}")


Context gốc:
Vào những năm 1870, hai nhà điêu khắc Augustus Saint-Gaudens và Daniel Chester French sinh sống và làm việc gần Quảng trường. Đến những năm 1920, Công viên Quảng trường Washington được công nhận cấp quốc gia là một trung tâm của phong trào nổi loạn về nghệ thuật và đạo đức. Do đó, khuôn viên NYU tại Quảng trường Washington trở nên đa dạng và hối hả nhờ có năng lượng của cuộc sống đô thị, điều này đã dẫn đến những sự thay đổi về mặt học thuật ở NYU. Những cư dân nổi tiếng thời kỳ này bao gồm Eugene O'Neill, John Sloan, và Maurice Prendergast. Vào những năm 1930, những người theo chủ nghĩa biểu hiện trừu tượng Jackson Pollock và Willem de Kooning, cùng với những người ủng hộ thuyết duy thực Edward Hopper và Thomas Hart Benton đều có xưởng vẽ xung quanh Quảng trường Washington. Những năm 1960 khu này trở thành một trong những trung tâm của thế hệ âm nhạc beat và folk (dân gian), khi Allen Ginsberg và Bob Dylan sinh sống tại đó. Điều này đã dẫn tới căng thẳng giữa họ và NYU, v